In [20]:
from module.setting import instCpStockCode, instCpCodeMgr, instStockChart
from module import connection, get, save, search, update
from datetime import datetime
from datetime import timedelta
from sqlalchemy import create_engine
import pandas as pd
import pymysql
import pandas as pd
import os

In [ ]:
# 종목 데이터 실시간
stock_list = ['A000020', 'A000040', 'A000050', 'A000060', 'A000070', 'A000100', 'A000120', 'A000140', 'A000150', 'A000180']

today = str(datetime.today().year) + str(datetime.today().month) + str((datetime.today()).day)



for stock_code in stock_list:
    get.get_stock_info(stock_code, today, today, 'm')

In [35]:
# mysql connect하기 위한 아이디 비밀번호 포트 데이터베이스 등록 및 conn 리턴
def sqlalchemy_connect_ip(ip_address, db_name):
    engine = create_engine("mysql+pymysql://admin:"
                +"big15" # user password
                +"@{0}:3306/{1}?charset=utf8".format(ip_address, db_name)
                , encoding='utf8')
    
    return engine.connect()

def get_pymysql_db_list(conn, db_name):

    # 현재 DB 내 존재하는 테이블(종목) 추출
    sql = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = '{0}'".format(db_name)

    result = conn.execute(sql)
    db_list = [item[0] for item in result.fetchall()]
    conn.close()

    return db_list

def lastest_investing_data(conn, date, investing_data_list, investing_df):

    # 현재 DB 내 존재하는 테이블(종목) 추출
    for table in investing_data_list:
        conn = sqlalchemy_connect_ip('192.168.50.123', 'investing_data')
        print(table)
        sql = "SELECT * FROM investing_data.`{0}` WHERE 날짜={1} ".format(table, date)

        result = conn.execute(sql)

        temp_df = pd.DataFrame(result.fetchall())
        investing_df = pd.concat([investing_df, temp_df], axis=1)
    
    conn.close()
    return investing_df

investing_df = pd.DataFrame()

sqlalchemy_conn = sqlalchemy_connect_ip('192.168.50.123', 'investing_data')

investing_data_list = get_pymysql_db_list(sqlalchemy_conn, 'investing_data')

# print(investing_data_list)

investing_df = lastest_investing_data(sqlalchemy_conn, '20221101', investing_data_list, investing_df)
investing_df


aedkrw내역
atx내역
audkrw내역
bse인도센섹스30내역
btckrwbithumb내역
cadkrw내역
cboevolatilityindex내역
chfkrw내역
cnykrw내역
dax내역
dkkkrw내역
dowjonesshanghai내역
ethkrwbithumb내역
eurkrw내역
ftsechinaa50내역
gbpkrw내역
germany3month채권수익율내역
hkdkrw내역
idrkrw내역
inrkrw내역
jpykrw내역
mxnkrw내역
myrkrw내역
nifty50내역
nokkrw내역
nzdkrw내역
pkrkrw내역
psi내역
rtsi지수내역
s&p500내역
s&pbmvipc내역
sarkrw내역
sekkrw내역
setindex내역
sgdkrw내역
szsecomponent내역
ta35내역
thbkrw내역
usdkrw내역
vn30내역
wti유선물내역
xrpkrwbithumb내역
구리선물내역
금선물내역
나스닥종합지수내역
네덜란드aex내역
닛케이내역
다우존스내역
대만가권내역
독일10년채권수익율내역
독일15년채권수익율내역
독일1년채권수익율내역
독일20년채권수익율내역
독일25년채권수익율내역
독일2년채권수익율내역
독일30년채권수익율내역
독일3개월채권수익율내역
독일3년채권수익율내역
독일4년채권수익율내역
독일5년채권수익율내역
독일6개월채권수익율내역
독일6년채권수익율내역
독일7년채권수익율내역
독일8년채권수익율내역
독일9개월채권수익율내역
독일9년채권수익율내역
러셀2000지수내역
러시아10년채권수익율내역
러시아15년채권수익율내역
러시아1개월채권수익율내역
러시아1년채권수익율내역
러시아1주채권수익율내역
러시아20년채권수익율내역
러시아2개월채권수익율내역
러시아2년채권수익율내역
러시아2주채권수익율내역
러시아3개월채권수익율내역
러시아3년채권수익율내역
러시아5년채권수익율내역
러시아6개월채권수익율내역
러시아7년채권수익율내역
러시아moexrussia내역
러시아오버나잇채권수익율내역
미국10년물국채금리채권수익율내역
미국1개월채권수익율내역
미국1년채권수익율내역
미국20년채권수익율내역
미국2년

,날짜,AEDKRW내역_종가,AEDKRW내역_오픈,AEDKRW내역_고가,AEDKRW내역_저가,AEDKRW내역_거래량,AEDKRW내역_변동,날짜,ATX내역_종가,ATX내역_오픈,...,항셍내역_저가,항셍내역_거래량,항셍내역_변동,날짜,호주S&PASX내역_종가,호주S&PASX내역_오픈,호주S&PASX내역_고가,호주S&PASX내역_저가,호주S&PASX내역_거래량,호주S&PASX내역_변동
0,20221101,385.66,388.52,389.2,384.36,0,-0.64,20221101,2949.37,2937.79,...,14859.92,3790000000,5.23,20221101,6976.9,6863.5,6976.9,6858.2,715850000,1.65


In [36]:
investing = investing_df.drop(columns='날짜', axis=1)

##############################

,AEDKRW내역_종가,AEDKRW내역_오픈,AEDKRW내역_고가,AEDKRW내역_저가,AEDKRW내역_거래량,AEDKRW내역_변동,ATX내역_종가,ATX내역_오픈,ATX내역_고가,ATX내역_저가,...,항셍내역_고가,항셍내역_저가,항셍내역_거래량,항셍내역_변동,호주S&PASX내역_종가,호주S&PASX내역_오픈,호주S&PASX내역_고가,호주S&PASX내역_저가,호주S&PASX내역_거래량,호주S&PASX내역_변동
0,385.66,388.52,389.2,384.36,0,-0.64,2949.37,2937.79,2980.39,2935.91,...,15632.85,14859.92,3790000000,5.23,6976.9,6863.5,6976.9,6858.2,715850000,1.65
